# *Initial* **Setup**

## **Library** *Settings*

In [ ]:
# Library Download
%pip install --upgrade pip
%pip install numpy
%pip install argparse
%pip install pandas
%pip install scipy
%pip install torch
%pip install sklearn
%pip install keras==2.10
%pip install tensorflow
%pip install matplot
%pip install plotly
%pip install h5py
%pip install h5pyViewer
%pip install pytorch_lightning
%pip install dipy
%pip install openpyxl
%pip install tabulate

In [ ]:
# Library Import
import os
import psutil
import numpy as np
import argparse
import pandas as pd
import scipy
import torch
import torch.nn as nn
import pytorch_lightning as pl
import sklearn
import keras
import tensorflow
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import h5py
import dipy
import warnings


In [274]:
# Functionality Import
from pathlib import Path
from typing import List, Literal, Optional, Callable, Dict, Literal, Optional, Union, Tuple
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from pytorch_lightning.utilities.cli import DATAMODULE_REGISTRY

#from keras.layers import Input, Dense, Lambda
#from keras.models import Model
#from keras import backend as K

from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input, ZeroPadding2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
tensorflow._api.v2.compat.v1.disable_v2_behavior()
from dipy.reconst.shm import cart2sphere, real_sh_descoteaux_from_index, sph_harm_ind_list
from PIL import Image
from tabulate import tabulate
warnings.filterwarnings('ignore')

## **Control** *Station*

In [ ]:
# Parser Initialization
parser = argparse.ArgumentParser(
    description = "MUDIVisualizer")

# Filepath Arguments
path = parser.add_argument_group('Required Filepaths')
path.add_argument('--data_filepath', type = Path, default = 'MUDI Dataset/data_.hdf5',
                    help = 'Filepath for DHDF5 File containing MUDI Dataset Values')
path.add_argument('--param_filepath', type = Path, default = 'MUDI Dataset/parameters_new.xlsx',
                    help = 'Filepath for DHDF5 File containing MUDI Dataset Parameters')
path.add_argument('--patient_filepath', type = Path, default = 'MUDI Dataset/header1_.csv',
                    help = 'Filepath for DHDF5 File containing MUDI Dataset Patient Information')
path.add_argument('--y_filepath', type = Path, default = 'MUDI Dataset/header_.csv',
                    help = 'Filepath for DHDF5 File containing MUDI Dataset Patient Information')

# ----------------------------------------------------------------------------------------------------------------------------

# Control Arguments for Datasets' Organization
mudi = parser.add_argument_group("MUDI Dataset's Control Parameters")
mudi.add_argument('--vShuffle', type = bool, default = False,
                    help = 'Control Variable for Vertical / Voxel Shuffle in Dataset')
mudi.add_argument('--hShuffle', type = bool, default = False,
                    help = 'Control Variable for Horizontal / Parameter Shuffle in Dataset')

# ----------------------------------------------------------------------------------------------------------------------------

# Control Arguments for CVAE Model's Parameters
cvae = parser.add_argument_group("CVAE Control Parameters")
cvae.add_argument('--batch_size', type = int, default = 500,
                    help = "Batch Size for DataLoaders")
cvae.add_argument('--nEpoch', type = int, default = 50,
                    help = "Number of Epochs for Model Training")
cvae.add_argument('--latentK', type = int, default = 50,
                    help = "Latent Space Dimensionality")
cvae.add_argument('--alpha', type = float, default = 0.001,
                    help = "Optimizers' Learning Rate Value")

# CVAE Version II - Keras Arguments
cvae_keras = cvae.add_argument_group("Keras V2")
cvae_keras.add_argument('--nnDim', type = int, default = 512,
                        help = "Encoder's Neural Network Dimensionality")

# ----------------------------------------------------------------------------------------------------------------------------

parse = parser.parse_args("")
parse.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **Data** *Access*

In [360]:
# MUDI Dataset Initialization Class
class MUDI(Dataset):

    # Constructor Function
    def __init__(
        self,
        dataset_filepath: Path,     # Path for File containing Dataset Voxel Values
        param_filepath: Path,       # Path for File containing all 1344 Parameter Settings Combination
        patient_filepath: Path,     # Path for File Containing Number of Voxels per Patients
        y_filepath: Path,           # Path for File Containing Voxels per Patient
    ):

        # Class Variable Logging
        super(MUDI).__init__()
        self.dataset_filepath = dataset_filepath
        self.param_filepath = param_filepath
        self.patient_filepath = patient_filepath
        self.y_filepath = y_filepath

        # Required Memory Space
        file_size = os.path.getsize(dataset_filepath)
        available_memory = psutil.virtual_memory().available
        assert(available_memory >= file_size
        ), f"ERROR: Dataset requires {file_size}b, but only {available_memory}b is available!"

        # Dataset's Parameter & Patient Info Access
        self.params = pd.read_excel(self.param_filepath)
        self.patientInfo = pd.read_csv(self.patient_filepath)
        self.patientInfo['Patient'].iloc[-1] = 'Total'
        self.y = pd.read_csv(self.y_filepath)
        self.y = pd.DataFrame(self.y['1'].values, columns = ['y'])
        self.num_patients = self.patientInfo.shape[0] - 1       # Total Number of Patients in Dataset
        self.num_params = self.params.shape[0]                  # Total Number of Parameters in Dataset

        # Dataset's Value Access
        if 'data' not in dir(self):
            print("Now Downloading Data...")
            with h5py.File(dataset_filepath, 'r') as dataset:
                self.data = pd.DataFrame(dataset.get('data1'))
                #self.data.columns = (self.params.T).values.tolist()
        

    # ///////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    # ///////////////////////////////////////////////////////////////////////////////////////////////////////////////////


    # Dataset Splitting Function
    def split(
        self,
        percentage: bool = False,       # Control Variable for the Format of Parameter Limits
        train_patients: int = 4,        # No. Patients used in Training
        train_params: int = 500,        # No. / Percentage of Parameters used in Training
        val_patients: int = 1,          # No. Patients used in Validation
        val_params: int = 10,           # No. / Percentage of Parameters used in Validation
    ):

        # General Assertions for Input Values
        print("Proceeding to Data Splitting...\n")
        assert( 0 < parse.batch_size <= self.num_params                       # Limits for Batch Size 
        ), f"ERROR: Requested Batch Size not Supported!"
        assert( 0 < train_patients <= self.num_patients                 # Limits for No. of Training Patients
        ), f"ERROR: Training Patient Number not Supported!"
        assert( 0 < val_patients <= self.num_patients                   # Limits for No. of Validation Patients
        ), f"ERROR: Validation Patient Number not Supported!"
        assert( train_patients + val_patients <= self.num_patients      # Limits for No. of Selected Patients
        ), f"ERROR: Total Patient Number not Supported!"
        assert( 0 < train_params <= self.num_params                     # Limits for No. of Training Parameters
        ), f"ERROR: Training Parameter Number not Supported!"
        assert( 0 < val_params <= self.num_params                       # Limits for No. of Validation Parameters
        ), f"ERROR: Validation Parameter Number not Supported!"

        # Split Datasets' Vertical Shape Initialization
        self.train_patients = train_patients
        self.train_samples = sum(self.patientInfo['Voxels'].iloc[0 : train_patients])
        self.val_patients = val_patients
        self.val_samples = sum(self.patientInfo['Voxels'].iloc[train_patients : train_patients + val_patients])
        self.test_patients = self.num_patients - (train_patients + val_patients)
        self.test_samples = sum(self.patientInfo['Voxels'].iloc[train_patients + val_patients : -1])
        
        # ----------------------------------------------------------------------------------------------------------------------------

        # Split Datasets' Vertical Shape Initialization (Percentage Value)
        if(percentage):
            
            # Assertions for Parameter Input Values
            assert( 0 < train_params <= 100                 # Percentage Limits for Training Parameters
            ), f"ERROR: Training Parameter Number not Supported!"
            assert(0 < val_params <= 100                    # Percentage Limits for Validation Parameters
            ), f"ERROR: Validation Parameter Number not Supported!"
            
            # Class Variable Logging
            self.trainTrain_params = train_params           # Percentage of Training Set's Training Parameters
            self.trainVal_params = 100 - train_params       # Percentage of Training Set's Validation Parameters
            self.valTrain_params = val_params               # Percentage of Validation Set's Training Parameters
            self.valVal_params = 100 - val_params           # Percentage of Validation Set's Validation Parameters
            if( self.test_patients != 0):
                self.testTrain_params = val_params                      # Percentage of Test Set's Training Parameters
                self.testVal_params = 100 - self.testTrain_params       # Percentage of Test Set's Validation Parameters
        
        # Split Datasets' Vertical Shape Initialization (Percentage Value)
        else:

            # Assertions for Parameter Input Values
            assert( 0 < train_params <= self.num_params                 # Percentage Limits for Training Parameters
            ), f"ERROR: Training Parameter Number not Supported!"
            assert(0 < val_params <= self.num_params                    # Percentage Limits for Validation Parameters
            ), f"ERROR: Validation Parameter Number not Supported!"
            
            # Class Variable Logging
            self.trainTrain_params = train_params                                   # Percentage of Training Set's Training Parameters
            self.trainVal_params = self.num_params - self.trainTrain_params         # Percentage of Training Set's Validation Parameters
            self.valTrain_params = val_params                                       # Percentage of Validation Set's Training Parameters
            self.valVal_params = self.num_params - self.valTrain_params             # Percentage of Validation Set's Validation Parameters
            if( self.test_patients != 0):
                self.testTrain_params = val_params                                  # Percentage of Test Set's Training Parameters
                self.testVal_params = self.num_params - self.testTrain_params       # Percentage of Test Set's Validation Parameter

        # ----------------------------------------------------------------------------------------------------------------------------

        # Dataset Vertical / Sample Splitting (Without Test Set)
        X_train, X_val, y_train, y_val = train_test_split(
            self.data, self.y,
            test_size = self.val_samples + self.test_samples,
            shuffle = False, random_state = 42)

        # Dataset Vertical / Sample Splitting (With Test Set)
        if(self.test_patients != 0):
            X_val, X_test, y_val, y_test = train_test_split(
                X_val, y_val,
                test_size = self.test_samples,
                shuffle = False, random_state = 42)

        # ----------------------------------------------------------------------------------------------------------------------------

        # Dataset Horizontal / Parameter Splitting | Training Set
        X_trainTrain, X_trainVal, self.params_trainTrain, self.params_trainVal = train_test_split(
                    X_train.T, self.params,
                    test_size = self.trainVal_params,
                    shuffle = parse.hShuffle, random_state = 42)
        self.X_trainTrain = pd.concat([X_trainTrain.T, y_train], axis = 1)
        self.X_trainVal = pd.concat([X_trainVal.T, y_train], axis = 1)
        
        # Dataset Horizontal / Parameter Splitting | Validation Set
        X_valTrain, X_valVal, self.params_valTrain, self.params_valVal = train_test_split(
                    X_val.T, self.params,
                    test_size = self.valVal_params,
                    shuffle = parse.hShuffle, random_state = 42)
        self.X_valTrain = pd.concat([X_valTrain.T, y_val], axis = 1)
        self.X_valVal = pd.concat([X_valVal.T, y_val], axis = 1)

        # Dataset Horizontal / Parameter Splitting | Testing Set
        if( self.test_patients != 0):
            X_testTrain, X_testVal, self.params_testTrain, self.params_testVal = train_test_split(
                        X_test.T, self.params,
                        test_size = self.testVal_params,
                        shuffle = parse.hShuffle, random_state = 42)
            self.X_testTrain = pd.concat([X_testTrain.T, y_test], axis = 1)
            self.X_testVal = pd.concat([X_testVal.T, y_test], axis = 1)
        
        # ----------------------------------------------------------------------------------------------------------------------------
        
        # DataLoader Building
        self.dl_trainTrain = DataLoader(torch.tensor(self.X_trainTrain.values, dtype = torch.float32),
                                        batch_size = parse.batch_size, shuffle = parse.vShuffle)
        self.dl_trainVal = DataLoader(torch.tensor(self.X_trainVal.values, dtype = torch.float32),
                                        batch_size = parse.batch_size, shuffle = parse.vShuffle)
        self.dl_valTrain = DataLoader(torch.tensor(self.X_valTrain.values, dtype = torch.float32),
                                        batch_size = parse.batch_size, shuffle = parse.vShuffle)
        self.dl_valVal = DataLoader(torch.tensor(self.X_valVal.values, dtype = torch.float32),
                                        batch_size = parse.batch_size, shuffle = parse.vShuffle)
        if (self.test_patients != 0):
            self.dl_testTrain = DataLoader(torch.tensor(self.X_testTrain.values, dtype = torch.float32),
                                        batch_size = parse.batch_size, shuffle = parse.vShuffle)
            self.dl_testVal = DataLoader(torch.tensor(self.X_testVal.values, dtype = torch.float32),
                                        batch_size = parse.batch_size, shuffle = parse.vShuffle)
                                        
        #Split Datasets' Contents Report
        if(percentage):
            print(tabulate([[self.train_patients, self.train_samples, f"{(self.trainTrain_params / 100) * self.num_params} ({np.round(self.trainTrain_params, 2)}%)", f"{(self.trainVal_params / 100) * self.num_params} ({np.round(self.trainVal_params, 2)}%)"],
                            [self.val_patients, self.val_samples, f"{(self.valTrain_params / 100) * self.num_params} ({np.round(self.valTrain_params, 2)}%)", f"{(self.valVal_params / 100) * self.num_params} ({np.round(self.valVal_params, 2)}%)"],
                            [self.test_patients, self.test_samples, f"{(self.valTrain_params / 100) * self.num_params} ({np.round(self.valTrain_params, 2)}%)", f"{(self.valVal_params / 100) * self.num_params} ({np.round(self.valVal_params, 2)}%)"],
                            [self.num_patients, (self.train_samples + self.val_samples + self.test_samples), "", ""]],
                            headers = ['No. Patients', 'No. Voxels', 'Training Parameters', 'Validation Parameters'],
                            showindex = ['Training Set', 'Validation Set', 'Test Set', 'Total'], tablefmt = 'fancy_grid'))
        else:
            print(tabulate([[self.train_patients, self.train_samples, f"{self.trainTrain_params} ({np.round((self.trainTrain_params / self.num_params) * 100, 2)}%)", f"{self.trainVal_params} ({np.round((self.trainVal_params   / self.num_params) * 100, 2)}%)"],
                            [self.val_patients, self.val_samples, f"{self.valTrain_params} ({np.round((self.valTrain_params / self.num_params) * 100, 2)}%)", f"{self.valVal_params} ({np.round((self.valVal_params   / self.num_params) * 100, 2)}%)"],
                            [self.test_patients, self.test_samples, f"{self.valTrain_params} ({np.round((self.valTrain_params / self.num_params) * 100, 2)}%)", f"{self.valVal_params} ({np.round((self.valVal_params   / self.num_params) * 100, 2)}%)"],
                            [self.num_patients, (self.train_samples + self.val_samples + self.test_samples), "", ""]],
                            headers = ['No. Patients', 'No. Voxels', 'Training Parameters', 'Validation Parameters'],
                            showindex = ['Training Set', 'Validation Set', 'Test Set', 'Total'], tablefmt = 'fancy_grid'))


In [361]:
# Dataset Initialization Example
data = MUDI(parse.data_filepath,        # Dataset Initialization
            parse.param_filepath,       # using Filepaths indicated
            parse.patient_filepath,     # in the Parser Arguments
            parse.y_filepath)
data.split()                            # Dataset Splitting using Default Values

# DataLoader Usage Example (Training Parameters for Training Set)
assert(len(data.dl_trainTrain) == int(np.ceil(data.train_samples / parse.batch_size)))
for i, batch in enumerate(data.dl_trainTrain):
    print(f"Batch #{i + 1}: {batch.shape[0]} Voxels / Samples | {batch.shape[1]} Settings / Parameters")


Now Downloading Data...
Proceeding to Data Splitting...

╒════════════════╤════════════════╤══════════════╤═══════════════════════╤═════════════════════════╕
│                │   No. Patients │   No. Voxels │ Training Parameters   │ Validation Parameters   │
╞════════════════╪════════════════╪══════════════╪═══════════════════════╪═════════════════════════╡
│ Training Set   │              4 │       468568 │ 500 (37.2%)           │ 844 (62.8%)             │
├────────────────┼────────────────┼──────────────┼───────────────────────┼─────────────────────────┤
│ Validation Set │              1 │       108378 │ 10 (0.74%)            │ 1334 (99.26%)           │
├────────────────┼────────────────┼──────────────┼───────────────────────┼─────────────────────────┤
│ Test Set       │              0 │            0 │ 10 (0.74%)            │ 1334 (99.26%)           │
├────────────────┼────────────────┼──────────────┼───────────────────────┼─────────────────────────┤
│ Total          │              5 

# **Model** Build

## **Version I** - *PyTorch*

### **Architecture** *Construction*

In [ ]:
class MUDIEncoder(nn.Module):

    # Constructor Function
    def __init__(
        self,
        inDim: int,
        latDim: int,
        weight: str = None,
    ) -> None:
        super(MUDIEncoder, self).__init__()

        # Class Variable Initialization
            # - The Latent Space Dimensions must be smaller than the Input Image's ;
            # - The image provided is necessarily Cubic (sum(data.shape)/3 = data.shape[0]);
        assert(latDim < inDim & inDim > 2
        ),f"The Dimensions provided are not Acceptable!"
        self.inDim = inDim
        self.latDim = latDim

        # Random Weight Initialization
        assert(weight in [None, 'Xavier Normal', 'Xavier Uniform']
        ), "ERROR: Non-Implemented Weight Initialization Method!"                  # Precaution for Non-Disclosed Weight Initialization Methods
        if weight is None:
            self.w = nn.init.orthogonal_()
        elif weight == 'Xavier Normal':
            self.w = nn.init.xavier_normal_(torch.empty(latDim, inDim))     # Normal Xavier Initialization Method
        elif weight == 'Xavier Uniform':
            self.w = nn.init.xavier_uniform_(torch.empty(latDim, inDim))    # Uniform Xavier Initialization Method
        self.w = nn.Parameter(self.w, requires_grad = True)                 # Ramdomly Initialized Weights upon Class First Call

        # Encoder Architecture
        self.encode = nn.Sequential(nn.Conv2d)

        # Forward Propagation Function
        #def forward(self, x: torch.Tensor) -> torch.Tensor:
        

In [ ]:
MUDIEncoder(5, 2, 'Uniform')

## **Version II** - *Keras*

### **Architecture** *Construction* ***(V1)***

This version utilizes Tensorflow's Model Class Batch iterator, unlike Version 2, to create the DataLoaders for both Training and Validation Data for the Training Set, hence the use of all Training Samples / Voxels.

In [342]:
# Data Dimensionality Definition
#X = Input(shape = (data.trainTrain_params, parse.batch_size))
X = Input(shape = (data.trainTrain_params, ))
y = Input(shape = ((data.params_trainTrain.shape[1] + 1), data.trainTrain_params))
input = concatenate([X, y[0]], axis = 0)

# Encoder Architecture
activ = 'relu'; optim = Adam(lr = parse.alpha)
encoder = Dense(parse.nnDim, activation = activ)(input)
mu = Dense(parse.latentK, activation  = 'linear')(encoder)
sigma = Dense(parse.latentK, activation = 'linear')(encoder)

In [2]:
# Latent Space Sampling
def sample(args):
    mu, sigma = args
    eps = K.random_normal(  shape = (parse.batch_size, parse.latentK), 
                            mean = 0.0, stddev = 1.0)
    return mu + K.exp(sigma / 2) * eps
z = tensorflow.transpose(Lambda(sample, output_shape = (parse.latentK, ))([mu, sigma]))
zc = tensorflow.transpose(concatenate([y[0], z], axis = 0))

NameError: name 'tensorflow' is not defined

In [345]:
# Decoder Architecture
decoder1 = Dense(data.trainTrain_params, activation = activ)
decoder2 = Dense(data.trainTrain_params, activation = 'sigmoid')
output = decoder2(decoder1(zc))

# Decoder Pipeline
decoderInput = Input(shape = (parse.latentK + (data.params_trainTrain.shape[1] + 1), ))
decoderOutput = decoder2(decoder1(decoderInput))
decoder = Model(decoderInput, decoderOutput)

In [346]:
# Loss Functions
def vae_loss(y, p):
    recon = K.sum(K.binary_crossentropy(y, p), axis = -1)
    kl = 0.5 * K.sum(K.exp(sigma) + K.square(mu) - 1.0 - sigma, axis=-1)
    return recon + kl

def KL_loss(y, p):
	return(0.5 * K.sum(K.exp(sigma) + K.square(mu) - 1.0 - sigma, axis = 1))

def recon_loss(y, p):
	return K.sum(K.binary_crossentropy(y, p), axis = -1)

In [347]:
#  Model Construction
cvae = Model([X, y], output)
encoder = Model([X, y], mu)
cvae.compile(optimizer = optim,
             loss = vae_loss,
             metrics = [KL_loss, recon_loss])

In [ ]:
# Model Training
cvae_hist = cvae.fit([data.X_trainTrain, data.params_trainTrain], data.X_trainTrain,
                    batch_size = parse.batch_size,
                    epochs = parse.nEpoch,
                    validation_data = ([data.X_trainVal, data.params_trainVal], data.X_trainVal),
                    callbacks = [EarlyStopping(patience = 5)])

### **Architecture** *Construction* ***(V2)***

In [ ]:
# Data Dimensionality Definition
X = Input(shape = (data.params_trainTrain.shape[0]))
y = Input(shape = (data.params_trainTrain.shape[1] + 1, data.params_trainTrain.shape[0]))
input = concatenate([X, y[0]], axis = 0)

# Encoder Architecture
activ = 'relu'; optim = Adam(lr = parse.alpha)
encoder = Dense(parse.nnDim, activation = activ)(input)
mu = Dense(parse.latentK, activation  = 'linear')(encoder)
sigma = Dense(parse.latentK, activation = 'linear')(encoder)

In [ ]:
# Latent Space Sampling
def sample(args):
    mu, sigma = args
    eps = K.random_normal(  shape = (parse.batch_size, parse.latentK), 
                            mean = 0.0, stddev = 1.0)
    return mu + K.exp(sigma / 2) * eps
z = tensorflow.transpose(Lambda(sample, output_shape = (parse.latentK, ))([mu, sigma]))
zc = concatenate([y[0], z], axis = 0)

In [ ]:
# Decoder Architecture
decoder1 = Dense(parse.nnDim, activation = activ)
decoder2 = Dense(parse.nnDim, activation = 'sigmoid')
output = decoder2(decoder1(zc))

# Decoder Pipeline
decoderInput = Input(shape = (parse.latentK + (data.params_trainTrain.shape[1] + 1), parse.batch_size))
decoderOutput = decoder2(decoder1(decoderInput[0]))
decoder = Model(decoderInput, decoderOutput)

In [ ]:
# Loss Functions
def vae_loss(y, p):
    recon = K.sum(K.binary_crossentropy(y, p), axis = -1)
    kl = 0.5 * K.sum(K.exp(sigma) + K.square(mu) - 1.0 - sigma, axis=-1)
    return recon + kl

def KL_loss(y, p):
	return(0.5 * K.sum(K.exp(sigma) + K.square(mu) - 1.0 - sigma, axis = 1))

def recon_loss(y, p):
	return K.sum(K.binary_crossentropy(y, p), axis = -1)

In [ ]:
#  Model Construction
cvae = Model([X, y], output)
encoder = Model([X, y], mu)
cvae.compile(optimizer = optim,
             loss = vae_loss,
             metrics = [KL_loss, recon_loss])

### **Training** *Mode* ***(V1)***

In [362]:
data.val_trainTrain

,Gradient x,Gradient y,Gradient z,b Value,TI,TE
0,-0.52602,-0.71803,-0.45578,3000,20.00,80
1,-0.52602,-0.71803,-0.45578,3000,1236.40,80
2,-0.52602,-0.71803,-0.45578,3000,2472.70,80
3,-0.52602,-0.71803,-0.45578,3000,3709.10,80
4,-0.92845,0.32525,-0.17943,1000,176.62,80
...,...,...,...,...,...,...
495,0.74733,0.56726,-0.34600,2000,4415.60,105
496,-0.21429,0.64014,0.73777,3000,883.12,105
497,-0.21429,0.64014,0.73777,3000,2119.50,105
498,-0.21429,0.64014,0.73777,3000,3355.90,105


In [359]:
X_trainTrain = data.X_trainTrain[:].iloc[]
X_trainTrain.shape

SyntaxError: invalid syntax (3991421500.py, line 1)

In [ ]:
# Model Training
cvae_hist = cvae.fit([data.X_trainTrain[], data.params_trainTrain], data.X_trainTrain,
                    batch_size = parse.batch_size,
                    epochs = parse.nEpoch,
                    validation_data = ([data.X_trainVal, data.params_trainVal], data.X_trainVal),
                    callbacks = [EarlyStopping(patience = 5)])

### **Training** *Mode* ***(V2)***

In [ ]:
# Batch Loop
for i, batch in enumerate(data.dl_trainTrain):

    # Skip for Different-Patient Batches
    if(batch[0][-1] != batch[-1][-1]):
        print(f"Batch # {i}: Skipping Different-Patient Batch")
        continue
    else:

        # Data Dimensionality Definition
        batch = torch.tensor_split(batch, (-1, data.trainTrain_params + 1), dim = 1)
        y_batch = batch[1]; X_batch = batch[0]
        params_batch = torch.tensor(np.hstack((data. params_trainTrain, y_batch)))
        print(f"Batch # {i}: X_batch.shape[0] Samples / Voxels")

        #
        cvae_hist = cvae.fit([X_batch, params_batch], X_batch,
                            #batch_size = parse.batchSize,
                            epochs = parse.nEpoch,
                            callbacks = [EarlyStopping(patience = 5)])

        # Single Iteration Control Condition
        if(i == 0): break


### **Result** *Evaluation*